<a href="https://colab.research.google.com/github/baigouy/notebooks/blob/master/EPySeg_build_or_train_a_model_or_further_train_pretrained_EPySeg_model_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Before you start

__Note: You are using this at your own risk__, the author cannot be help responsible for damage to Google drive or misfunction of the e-mail account (due to drive overfilling). To avoid issues please use a second account dedicated to deep learning. __If you disagree, please quit this page__.

[General colab notebook tips](https://github.com/baigouy/notebooks#getting-started)

Please report bugs to baigouy@gmail.com

# Step 1:
- Mount Google drive ([How to](https://github.com/baigouy/notebooks#how-to-mount-google-drive-in-colab)).
- Select a GPU runtime ([How to](https://github.com/baigouy/notebooks#select-a-gpu-runtime)).
- Then run the code cell below ([How to](https://github.com/baigouy/notebooks#how-to-run-a-code-cell)).

In [1]:
#@title <-- Press Run
from __future__ import print_function # bug fix --> import future must be at the beginning
%tensorflow_version 2.x
import tensorflow as tf
import sys
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import Markdown, display
import os

# defining an html/md formatting print function
def printmd(string):
    display(Markdown(string))

# defining a filebrowser that can be used later on
# FileBrowser modified from DrDub https://gist.github.com/DrDub/6efba6e522302e43d055 PD licence
# new beahviour: single click selects a file or a folder, double click browses folders. Blocked browsing of files as if they were folders. Has an option to show files or not (files are shown by default).
# Note that I have made a very quick n dirty hack of the initial nice code so that it handles double click, my changes would need some love but it seems functional, please report bugs to BA otherwise
# TODO also maybe hack it to select files only or folders or maybe ignore and do the check later and put a warning accordingly (maybe simpler for now)


import os
import ipywidgets as widgets
from ipywidgets import Layout, Button, VBox, Label,Box, HBox, ButtonStyle
from timeit import default_timer as timer

class FileBrowser(object):
    def __init__(self, width='auto', height="200px", item_width="auto", item_height="auto", folders_only=False):
        
        self.folders_only = folders_only
        self.path = os.getcwd()
        self._update_files()
        
        self.layout = Layout(overflow='scroll',
                             border ='1px solid black',
                             width  ='{}'.format(width),
                             height ='{}'.format(height),
                             flex_flow = "column wrap",
                             align_items = "flex-start",
                             display='flex')
        
        self.button_dir_syle  = ButtonStyle(button_color='lightgray')
        self.button_file_syle = ButtonStyle(button_color='#Fafaff')
        
        self.button_layout    = Layout(left="0px", width="{}".format(item_width), height="{}".format(item_height))
        self.last_click = None
        self.root_folder = None
        
        
    def _update_files(self):
        self.files = list()
        self.dirs  = list()
        # print('is dir', self.path, os.path.isdir(self.path))
        if(os.path.isdir(self.path)):
            for f in os.listdir(self.path):
                ff = os.path.join(self.path,f)
                if os.path.isdir(ff):
                    self.dirs.append(f)
                else:
                    self.files.append(f)
        
    def widget(self):
        list_box = widgets.Box(layout=self.layout)
        box = VBox([list_box, Label(self.path)])
        self._update_box(box)
        
        return box
    
    
    def _update_box(self, main_box, double_click=True):
        # print('double click in there', double_click)
        path_label = main_box.children[1]
        file_or_path = "Selected file: {}"
        if os.path.isdir(self.path):
          file_or_path = "Selected path: {}"
        path_label.value = file_or_path.format(self.path)
        
        if self.root_folder is None:
          self.root_folder = self.path

        box        = main_box.children[0]
        if not double_click:
          # print('skipping')
          return box
        else:
          if (os.path.isfile(self.path)):
            # prevent 'browsing' files
            return box
          # print('starting path', self.path)
          self.root_folder = self.path

        def on_click(b):
            double_click = False

            # detect single or double click and act accordingly (dirty hack by BA)
            if self.last_click is not None:
              # print('delay',timer()-self.last_click)
              if timer()-self.last_click<=0.600:
                # print('double click')
                double_click = True
              else:
                # print('single click')
                self.last_click = timer()
            else:
              self.last_click = timer()  
              # print('single click')          

            if b.description == '..':
                if not double_click: #skip double click for previous folder
                  self.path = os.path.split(self.root_folder)[0]

                double_click=True
                # print('path parent',self.path)
            else:
                if not double_click:
                  self.path = os.path.join(self.root_folder, b.description)
                else:
                  self.path = os.path.join(self.root_folder, b.description)

            self._update_files()
            self._update_box(main_box, double_click)

        buttons = []
        if os.path.dirname(self.path) != self.path:
            button = widgets.Button(description='..', style=ButtonStyle(button_color='lightblue'), layout=self.button_layout)
            button.on_click(on_click)
            buttons.append(button)
        
        for f in self.dirs:
            button = widgets.Button(description=f, style=self.button_dir_syle, icon='fa-folder', tooltip=f, layout=self.button_layout)
            button.on_click(on_click)
            buttons.append(button)
        
        if not self.folders_only:
          for f in self.files:
              button = widgets.Button(description=f, style=self.button_file_syle, tooltip=f, layout=self.button_layout)
              button.on_click(on_click)
              buttons.append(button)
            
        box.children = buttons


try:
  # check that google drive is successfully mounted
  
  if os.path.exists('/content/drive/My Drive'):#/content/dri #create an error # /content/drive/My Drive # full path
    printmd('<font color="green">Google drive successfully mounted!</font>')
    # move to the drive folder
    %cd /content/drive/My Drive
  else:
    from google.colab import drive
    printmd('<font color="green"><b>Open the external link and follow the instructions until you get the authorization code. Copy this code, paste it below then press "Enter"</b></font>')
    drive.mount('/content/drive')

    if not os.path.exists('/content/drive/My Drive'):
      printmd('<font color="red">Please connect to Google drive and repeat this step:</font>')
      print('How to: https://github.com/baigouy/notebooks#how-to-mount-google-drive-in-colab') # could be useful indeed
      #assert False
      raise Exception('error!')
    else:
      # move to the drive folder
      %cd /content/drive/My Drive
      printmd('<font color="green">Google drive successfully mounted!</font>')
    # sys.exit()
    #quit(keep_kernel=False)


  # check we are using GPU
  device_name = tf.test.gpu_device_name()
  if device_name != '/device:GPU:0':
    printmd('<font color="red">GPU not found, please repeat this step.</font>')
    print('How to: https://github.com/baigouy/notebooks#select-a-gpu-runtime') # could be useful indeed
    # should also do that first and print the link to help people use this stuff!!!
    # quit
    raise Exception('error!')
  else:
    printmd('<font color="green">GPU found!</font>')
    # should I merge all in one single step because if anyway anything fails it is not gonna work...
    # maybe it's an idea

  printmd('<br><font color="green">Everything went fine, please move on to next step!</font>')
except:
  pass


# printmd('this is a test')

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


<font color="green"><b>Open the external link and follow the instructions until you get the authorization code. Copy this code, paste it below then press "Enter"</b></font>

Mounted at /content/drive
/content/drive/My Drive


<font color="green">Google drive successfully mounted!</font>

<font color="green">GPU found!</font>

<br><font color="green">Everything went fine, please move on to next step!</font>

# Step 2:
- Install the required python libraries by running the code cell below (please be patient, this may take time...)

In [2]:
#@title <-- Press Run

# slow --> keep it as a separate step
# install python libraries necessary to run epyseg
# TODO check that all those libs are reallyu required but ok for now
!pip install czifile
!pip install h5py
!pip install Markdown
!pip install matplotlib
!pip install numpy
!pip install numpydoc
!pip install Pillow
!pip install PyQt5
!pip install PyQtWebEngine
!pip install read-lif
!pip install scikit-image
!pip install scipy
!pip install segmentation-models==1.0.1
!pip install tifffile>=2021.11.2
!pip install qtawesome
!pip install natsort
!pip install numexpr
!pip install elasticdeform
!pip install roifile
!pip install prettytable
!pip install pyperclip
!pip install scikit-learn
!pip install --no-deps epyseg # prevent reinstalling tf 2.x and rather use the google optimized tf
# !pip install --no-deps --index-url https://test.pypi.org/simple/ epyseg


# libraries loaded checking epyseg to see if everything is functional
try:
  from epyseg.img import Img
  # just try import any class from Epyseg --> will raise an error if loading fails if loads most likely everything should work
  from epyseg.deeplearning.deepl import EZDeepLearning
  from epyseg.deeplearning.augmentation.meta import MetaAugmenter
  from epyseg.deeplearning.augmentation.generators.data import DataGenerator
  deepTA = EZDeepLearning()
  printmd('<br><font color="green">EPySeg succesfully loaded, please move on to next step!</font>') 
except:
  printmd('<br><font color="red">EPySeg failed to load. Please repeat this step</font>.')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 48.8 MB/s eta 0:00:00
  Attempting uninstall: docutils
    Found existing installation: docutils 0.16
    Uninstalling docutils-0.16:
      Successfully uninstalled docutils-0.16
  Attempt

<br><font color="green">EPySeg succesfully loaded, please move on to next step!</font>

# Step 3
- Run the cell below and choose between loading or building a model, then move to next cell.

In [3]:
#@title <-- Press Run

label = widgets.Label('Please make a choice:')
# cannot be horizontal https://github.com/jupyter-widgets/ipywidgets/issues/1247
options = [['Build a new model',1], ['Load an existing model',2], ['Further train the default EPySeg model',3] ] #,['Use a pre-trained model',3]
model = widgets.RadioButtons(options=options)

model_version = widgets.Dropdown(
    value='v2',
    placeholder='Model version (pretrained only)',
    options=['v2', 'v1'],
    description='Model version (pretrained only)',
    ensure_option=True,
    disabled=False,
    layout={'width': 'max-content'},
    style={'description_width': 'initial'}
)

ui = widgets.VBox([label, model,model_version])
display(ui) 

# Step 4
- Please run the cell below and follow output instructions

In [4]:
#@title <-- Press Run

# print(model.value)
if model.value == 1:
  # print('build')
  # should offer loading a model from drive
  # should I allow to locate a file ????

  label_mod_params = Label('Please select model parameters (then move to next step):')
  # get all available model architectures
  available_architectures = deepTA.available_model_architectures
  architecture = widgets.Dropdown(
      value='Linknet',
      placeholder='Choose an architecture',
      options=available_architectures,
      description='Architecture',
      ensure_option=True,
      disabled=False,
      layout={'width': 'max-content'},
      style={'description_width': 'initial'}
  )

  # get all available backbones
  available_backbones = deepTA.available_sm_backbones
  backbone = widgets.Dropdown(
      value='vgg16',
      placeholder='Choose a backbone',
      options=available_backbones,
      description='Backbone',
      ensure_option=True,
      disabled=False,
      layout={'width': 'max-content'},
      style={'description_width': 'initial'}
  )

  # get all available activation for the last layer
  available_activations = deepTA.last_layer_activation
  activation = widgets.Dropdown(
      value='sigmoid',
      placeholder='Choose an activation',
      options=available_activations,
      description='Activation',
      ensure_option=True,
      disabled=False,
      layout={'width': 'max-content'},
      style={'description_width': 'initial'}
  )

  # model input width
  input_width = widgets.IntSlider(description='Width (0 = None = any size)', value=0, min=0, max=1024, step=2, style={'description_width': 'initial'})
  # model input height
  input_height = widgets.IntSlider(description='Height (0 = None = any size)', value=0, min=0, max=1024, step=2, style={'description_width': 'initial'})
  # model input channels
  inp_channels = widgets.IntSlider(description='Input channels', min=1, step=1, style={'description_width': 'initial'})
  # model output channels
  classes = widgets.IntSlider(description='Classes', min=1, step=1, style={'description_width': 'initial'})
  ui = widgets.VBox([label_mod_params, architecture, backbone, input_width, input_height, inp_channels,activation, classes])
  display(ui) 
elif model.value == 2:
  # print('load')
  # browse for input model
  load_model_label = Label('Please select a model file and move on to next step:')
  model_path = FileBrowser(folders_only=False) #item_width="140px"
  ui = widgets.VBox([load_model_label, model_path.widget()])
  display(ui)
elif model.value == 3:
  # make it load the default epyseg model and be ready to retrain it
  available_pretrained_models = deepTA.get_available_pretrained_models()
  printmd('<br><font color="green">Please move on to next step!</font>')

# else: # TODO deactivate for now!!!
  # print('pre-trained')

# Step 5
- Please run the cell below and follow instructions

In [5]:
#@title <-- Press Run

deepTA.model = None # does it really reset model or not
try:
  action = 'built'
  # First we try to build or load the model and if something goes wrong ask to repeat step 4
  if model.value == 1:
    # build model from scratch
    

    # architecture, backbone, input_width, input_height, inp_channels,activation, classes
    deepTA.load_or_build(architecture=architecture.value, backbone=backbone.value, activation=activation.value, classes=classes.value, input_width=input_width.value, input_height=input_height.value, input_channels=inp_channels.value)
    # deepTA.summary()
    # if deepTA.model is None:
      # printmd('<font color="green">Model successfully built!</font>')
    # else:
      # printmd('<font color="red">Could not build model, please check the model parameters. Repeat Step 4</font>')
  elif model.value == 2:
    # try to load an existing model and report error if model could not be loaded
    action = 'loaded'
    # check wether model can be loaded 

    path = model_path.path
    if not os.path.exists(path):
      printmd('<font color="red">Please provide a valid model path. </font>')
      raise Exception('error!')
    else:
      # try load the model and return error upon failure
      # printmd('<font color="red">test!</font>')
      deepTA.load_or_build(model=path)
  elif model.value == 3:
    action = 'loaded'
    # Load a pre-trained model
    #pretrained_model_parameters = deepTA.pretrained_models_2D_epithelia['Linknet-vgg16-sigmoid']
    pretrained_model_parameters = deepTA.pretrained_models['Linknet-vgg16-sigmoid'] if model_version.value=='v1' else  deepTA.pretrained_models[
                'Linknet-vgg16-sigmoid'+'-'+model_version.value]
    pretrained_model_name = 'Linknet-vgg16-sigmoid'  if model_version.value=='v1' else 'Linknet-vgg16-sigmoid'+'-'+model_version.value
    # print(pretrained_model_parameters)
    # deepTA.load_or_build(model=pretrained_model_parameters['model'], model_weights=pretrained_model_parameters['model_weights'], architecture=pretrained_model_parameters['architecture'], backbone=pretrained_model_parameters['backbone'], activation=pretrained_model_parameters['activation'], classes=pretrained_model_parameters['classes'], input_width=pretrained_model_parameters['input_width'], input_height=pretrained_model_parameters['input_height'], input_channels=pretrained_model_parameters['input_channels'],pretraining=pretrained_model_name)
    deepTA.load_or_build(architecture='Linknet', backbone='vgg16', activation='sigmoid', classes=7, pretraining=pretrained_model_name)

      
  if deepTA.model is None:
    printmd('<font color="red">Empty model!</font>')
    raise Exception('error!')
  else:
    deepTA.summary()
    printmd('<font color="green">Model succesfully '+ action +', please move on to next step!</font>')

except:
  printmd('<font color="red">Something went wrong, please repeat step 4</font>')
  pass

Model: "Linknet-vgg16-sigmoid"
______________________________________________________________________________________________________________________________________________________
 Layer (type)                                    Output Shape                     Param #           Connected to                                      
 input_1 (InputLayer)                            [(None, None, None, 1)]          0                 []                                                
                                                                                                                                                      
 block1_conv1 (Conv2D)                           (None, None, None, 64)           640               ['input_1[0][0]']                                 
                                                                                                                                                      
 block1_conv2 (Conv2D)                           (None, None, N

INFO - 2023-05-04 09:50:03,374 - deepl.py - summary - line 860 - None

INFO:master:None


<font color="green">Model succesfully built, please move on to next step!</font>

# Step 6
- Run the cell below and decide whether or not to load model weights then move to next step

In [6]:
#@title <-- Press Run
if model.value != 3:

  f = FileBrowser(folders_only=False) #item_width="140px"
  options2 = [['No',2],['Yes',1] ] #,['Use a pre-trained model',3]    
  drop_load_weights = widgets.Dropdown(options=options2, description='Load model weights:', layout={'width': 'max-content'}, style={'description_width': 'initial'})

  def browse_for_weights(x):
      if x == 1:
        weight_label = Label('Please select a weight file in your drive, then move on to next step:')
        ui = widgets.VBox([weight_label, f.widget()])
        display(ui)
      else:
        weight_label = Label('No weights to load, then please move on to next step')
        display(weight_label)


  interact(browse_for_weights, x=drop_load_weights);
else:
  printmd('<br><font color="green">Please move on to next step!</font>')



interactive(children=(Dropdown(description='Load model weights:', layout=Layout(width='max-content'), options=…

# Step 8
- Run the cell below and follow the instructions

In [7]:
#@title <-- Press run

if model.value != 3:
  if drop_load_weights.value == 1:
    try:
      deepTA.load_weights(f.path)
      printmd('<font color="green">Weights succesfully loaded. Please move on to next step!</font>')
    except:
      printmd('<font color="red">Something went wrong (file corrupt/wrong file), please repeat step 7.</font>')
  else:
    printmd('<font color="green">Please move on to next step!</font>')
else:
  printmd('<font color="green">Please move on to next step!</font>')    

<font color="green">Please move on to next step!</font>

# Step 9
- Run the cell to verify model compilation or set compilation parameters

In [8]:
#@title <-- Press Run

# check if already compiled and if so ask wether to go on or not or force recompile

# to choose the network optimizer, loss and metric

# get available optimizers
available_optimizers = deepTA.optimizers
optimizers = widgets.Dropdown(
    value='adam',
    placeholder='Choose an optimizer',
    options=available_optimizers,
    description='Optimizer',
    ensure_option=True,
    disabled=False,
    layout={'width': 'max-content'},
    style={'description_width': 'initial'}
)

default_loss = 'jaccard_loss'
if model.value == 3:
  default_loss = 'bce_jaccard_loss'

# get available losses
available_losses = deepTA.loss.keys()
losses = widgets.Dropdown(
    value=default_loss,
    placeholder='Choose a loss function',
    options=available_losses,
    description='Loss function',
    ensure_option=True,
    disabled=False,
    layout={'width': 'max-content'},
    style={'description_width': 'initial'}
)

# get available metrics
available_metrics = deepTA.metrics.keys()
metrics = widgets.Dropdown(
    value='iou_score',
    placeholder='Choose a metric',
    options=available_metrics,
    description='Metric',
    ensure_option=True,
    disabled=False,
    layout={'width': 'max-content'},
    style={'description_width': 'initial'}
)


default_lr_checkbox = widgets.Checkbox(
    value=True,
    description='Use default learning rate',
    disabled=False,
    indent=False,
    style={'description_width': 'initial'}
)

learning_rate_spinner = widgets.FloatSlider(
    value=0.001,
    min=0.0000001,
    max=0.10,
    step=0.00001,
    description='Learning rate:',
    disabled=True,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.7f',
    # layout={'width': 'max-content'},
    style={'description_width': 'initial'}
)
learning_rate_spinner.layout.visibility = 'hidden'


def show_hide_lr(x):
      # print(default_lr_checkbox.value)
      learning_rate_spinner.disabled = default_lr_checkbox.value
      if default_lr_checkbox.value:
        learning_rate_spinner.layout.visibility = 'hidden'
      else:
        learning_rate_spinner.layout.visibility = 'visible'

interactive(show_hide_lr, x=default_lr_checkbox);

force_recompile = not deepTA.is_model_compiled()
ui = widgets.VBox([losses, metrics, optimizers, default_lr_checkbox, learning_rate_spinner])

options2 = [['No',2],['Yes',1]] #,['Use a pre-trained model',3]    
force_recompile_drop = widgets.Dropdown(options=options2, description='Force recompile:', layout={'width': 'max-content'}, style={'description_width': 'initial'})  




def recompile_model(x):
    if x == 1:
      # model is already compiled offer a recompile
      # show a stuff to choose wether or not to handle that
      # force_recompile = True
      # print(force_recompile)
      display(ui)
    else:
      weight_label = Label('Model will not be recompiled')
      # force_recompile = False
      # print(force_recompile)
      display(weight_label)

if force_recompile:
  # force_recompile = True
  print('Model must be compiled!')
  # print(force_recompile)
  display(ui)
else:
  print('Model is already compiled')
  interact(recompile_model, x=force_recompile_drop);


Model must be compiled!


# Step 10
- Run the cell below and follow the instructions

In [9]:
#@title <-- Press Run

# TODO offer learning rate at som point...
# print(force_recompile, force_recompile_drop.value)
if force_recompile or   force_recompile_drop.value ==1:
  # do recompile the model
  try:
    deepTA.compile(optimizer=optimizers.value, loss=losses.value, metrics=[metrics.value])
    if not default_lr_checkbox.value:
      deepTA.set_learning_rate(learning_rate_spinner.value)
    # print(deepTA.is_model_compiled())
    # print(deepTA.get_loaded_model_params())
    printmd('<font color="green">Model successfully compiled, please move on to next step!</font>')
  except:
    printmd('<font color="red">Model compilation failed, please repeat previous steps.</font>')
else:
  printmd('<font color="green">Please move on to next step!</font>')

<font color="green">Model successfully compiled, please move on to next step!</font>

# Step 11
- Please choose a folder with the training images (raw images you want to segment)

In [10]:
#@title <-- Press Run

folder_train_originals = FileBrowser(folders_only=True)
folder_train_originals_label = Label('Please select the folder in your drive that contains the original images to use for training (then move on to next step):')
ui = widgets.VBox([folder_train_originals_label, folder_train_originals.widget()])
display(ui)

# Step 12
- Please choose a folder with the segmented images the model should try to reproduce

In [11]:
#@title <-- Press Run

folder_train_segmentation = FileBrowser(folders_only=True)
folder_train_segmentation_label = Label('Please select the folder in your drive that contains the segmented images corresponding to the original loaded in the previous cell (then move on to next step):')
ui = widgets.VBox([folder_train_segmentation_label, folder_train_segmentation.widget()])
display(ui)

# Step 13
-Run the cell below to check that training images and their corresponding segmentation masks are valid and as numerous 

In [13]:
#@title <-- Press Run

# print(folder_train_originals.path)
# print(folder_train_segmentation.path)

originals_list = DataGenerator.get_list_of_images(folder_train_originals.path)
# print(originals_list)
segmented_list = DataGenerator.get_list_of_images(folder_train_segmentation.path)
# print(segmented_list)    
print('originals n=',len(originals_list), 'segmented images n=', len(segmented_list))

first_orig = None
first_mask = None

if len(originals_list) > 0 and len(segmented_list)> 0 and len(originals_list) == len(segmented_list):
  try:
    print('first original image')
    first_orig = Img(originals_list[0])
    # first_orig.pop()
    print(first_orig.shape)
    print('first mask image')
    first_mask = Img(segmented_list[0])
    # first_mask.pop()
    print(first_mask.shape)
  
    printmd('<font color="green">Everything seems fine, please move on to next step.</font>')
  except:
    printmd('<font color="red">Problems detected. The first image(s) could not be loaded. Please check your files and repeat steps 11 to 13.</font>')    
else:
  if len(originals_list) != len(segmented_list):
    printmd('<font color="red">Problems detected. There seems to be a discrepancy between the number of original images and their corresponding segmentation. You may proceed but training will most likely fail. It is recommended you repeat steps 11 and 12 to change folders or check your files.</font>')
  else:
    printmd('<font color="red">Problems detected. A folder seems empty. Please repeat steps 11 to 13.</font>')


originals n= 6 segmented images n= 6
first original image
(512, 512, 3)
first mask image
(512, 512, 3)


<font color="green">Everything seems fine, please move on to next step.</font>

# Step 14
- Please run the cell below to define normalization and clip model input images (original images)

In [14]:
#@title <-- Press Run

# Normalization for orig data

info_label_orig=widgets.Label('If you don\'t know what to do, just leave pamaters unchanged and move on to next step.')

# if image has no or 1 channel then ignore COI
channels_orig = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
if first_orig is not None:
  if first_orig.has_c():
    channels_orig = []
    for c in range(first_orig.get_dimension('c')):
      channels_orig.append(c)
  else:
    channels_orig = [0]

channel_COI_orig = widgets.Dropdown(
    placeholder='Choose a channel of interest',
    options=channels_orig,
    description='Channel of interest (COI)',
    ensure_option=True,
    disabled=False,
    layout={'width': 'max-content'},
    style={'description_width': 'initial'}
)

rule_reduc_orig = widgets.Dropdown(
    placeholder='Rule to reduce nb of channels (if needed)',
    options=['copy the COI to all available channels','force copy the COI to all available channels even if nb of channels is ok','remove extra channels'],
    description='Rule to reduce nb of channels (if needed)',
    ensure_option=True,
    disabled=False,
    layout={'width': 'max-content'},
    style={'description_width': 'initial'}
)

rule_aug_orig = widgets.Dropdown(
    placeholder='Rule to increase nb of channels (if needed)',
    options=['copy the COI to all channels', 'force copy the COI to all available channels even if nb of channels is ok', 'copy the COI to missing channels only', 'add empty channels (0 filled)'],
    description='Rule to increase nb of channels (if needed)',
    ensure_option=True,
    disabled=False,
    layout={'width': 'max-content'},
    style={'description_width': 'initial'}
)


# TODO REMOVE LINE BELOW AND UPDATE LIBRARY!!!!
# ̣̣̣!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Img.clipping_methods = ['ignore outliers', '+', '+/-', '-']

# list of available normalization methods
# print(Img.normalization_methods)
# Img.normalization_ranges =[[0, 1], [-1, 1]]
# print('a',Img.normalization_ranges)
ranges = Img.normalization_ranges.copy()
# print(ranges)

for n,r in enumerate(ranges):
  # print(r,n, str(r))
  ranges[n]=str(r)

# print(ranges)

normalization_methods_orig = widgets.Dropdown(
    # value='Rescaling (min-max normalization)',
    placeholder='Normalization',
    options=Img.normalization_methods,
    description='Normalization method',
    ensure_option=True,
    disabled=False,
    layout={'width': 'max-content'},
    style={'description_width': 'initial'}
)
normalization_range_orig = widgets.Dropdown(
    # value='[0, 1]',
    placeholder='Range',
    options=ranges,
    description='Normalization range',
    ensure_option=True,
    disabled=False,
    # layout={'width': 'max-content'},
    style={'description_width': 'initial'}
)
clipping_methods_orig = widgets.Dropdown(
    placeholder='Clipping outliers',
    options=Img.clipping_methods,
    description='clip intensity',
    ensure_option=True,
    disabled=False,
    layout={'width': 'max-content'},
    style={'description_width': 'initial'}
)

per_channel_orig = widgets.Checkbox(True,
                       description='Per channel normalization',
                       style={'description_width': 'initial'}
)

cliprange_orig = widgets.FloatSlider(description='range (clipping)', value=0.005, min=0, max=0.30, step=0.001,style={'description_width': 'initial'})

percentile_min_orig = widgets.FloatSlider(description='min percentile', value=2., min=0., max=30., step=0.1,style={'description_width': 'initial'})
percentile_max_orig = widgets.FloatSlider(description='max percentile', value=99.8, min=70., max=100., step=0.1,style={'description_width': 'initial'})
clip_percentile_orig_checkbox = widgets.Checkbox(
    value=False,
    description='Clip',
    disabled=False,
    indent=False,
    style={'description_width': 'initial'}
)

percentile_min_orig.layout.visibility = 'hidden'
percentile_max_orig.layout.visibility = 'hidden'
clip_percentile_orig_checkbox.layout.visibility = 'hidden'


def display_percentile_range(x):
  if not 'ercent' in normalization_methods_orig.value:
    percentile_min_orig.layout.visibility = 'hidden'
    percentile_max_orig.layout.visibility = 'hidden'
    clip_percentile_orig_checkbox.layout.visibility = 'hidden'
    normalization_range_orig.layout.visibility = 'visible'
  else:
    percentile_min_orig.layout.visibility = 'visible'
    percentile_max_orig.layout.visibility = 'visible'
    clip_percentile_orig_checkbox.layout.visibility = 'visible'
    normalization_range_orig.layout.visibility = 'hidden'

interactive(display_percentile_range, x=normalization_methods_orig);
# display(per_channel)
# def changed(b):
#     print(b)

# per_channel.observe(changed)

# wdgts = []
# ui = widgets.VBox(wdgts)
# display(ui)

invert_orig = widgets.Checkbox(False,
                       description='Negative/invert intensity',
                       style={'description_width': 'initial'}
)


# print(range_value)
# print(per_channel.value) #recover checkbox value
ui = widgets.VBox([info_label_orig, widgets.HBox([clipping_methods_orig,  cliprange_orig]), channel_COI_orig, rule_reduc_orig, rule_aug_orig,  widgets.HBox([normalization_methods_orig, normalization_range_orig, per_channel_orig]),widgets.HBox([percentile_min_orig, percentile_max_orig, clip_percentile_orig_checkbox]),  invert_orig])
display(ui)



# Step 15
- Please run the cell below to define normalization and pre process segmented images (mask images)

In [15]:
#@title <-- Press Run

# Normalization for masks
# avec plus ou moins de preprocess style dilat

info_label_mask=widgets.Label('If you don\'t know what to do, just leave pamaters unchanged and move on to next step.')

# if image has no or 1 channel then ignore COI
channels_mask = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
if first_mask is not None:
  if first_mask.has_c():
    channels_mask = []
    for c in range(first_mask.get_dimension('c')):
      channels_mask.append(c)
  else:
    channels_mask = [0]

channel_COI_mask = widgets.Dropdown(
    placeholder='Choose a channel of interest',
    options=channels_mask,
    description='Channel of interest (COI)',
    ensure_option=True,
    disabled=False,
    layout={'width': 'max-content'},
    style={'description_width': 'initial'}
)

rule_reduc_mask = widgets.Dropdown(
    placeholder='Rule to reduce nb of channels (if needed)',
    options=['copy the COI to all available channels','force copy the COI to all available channels even if nb of channels is ok','remove extra channels'],
    description='Rule to reduce nb of channels (if needed)',
    ensure_option=True,
    disabled=False,
    layout={'width': 'max-content'},
    style={'description_width': 'initial'}
)

rule_aug_mask = widgets.Dropdown(
    placeholder='Rule to increase nb of channels (if needed)',
    options=['copy the COI to all channels', 'force copy the COI to all available channels even if nb of channels is ok', 'copy the COI to missing channels only', 'add empty channels (0 filled)'],
    description='Rule to increase nb of channels (if needed)',
    ensure_option=True,
    disabled=False,
    layout={'width': 'max-content'},
    style={'description_width': 'initial'}
)


# TODO REMOVE LINE BELOW AND UPDATE LIBRARY!!!!
# ̣̣̣!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Img.clipping_methods = ['ignore outliers', '+', '+/-', '-']

# list of available normalization methods
# print(Img.normalization_methods)
# Img.normalization_ranges =[[0, 1], [-1, 1]]
# print('a',Img.normalization_ranges)
ranges = Img.normalization_ranges.copy()
# print(ranges)

for n,r in enumerate(ranges):
  # print(r,n, str(r))
  ranges[n]=str(r)

# print(ranges)

normalization_methods_mask = widgets.Dropdown(
    # value='Rescaling (min-max normalization)',
    placeholder='Normalization',
    options=Img.normalization_methods,
    description='Normalization method',
    ensure_option=True,
    disabled=False,
    layout={'width': 'max-content'},
    style={'description_width': 'initial'}
)
normalization_range_mask = widgets.Dropdown(
    # value='[0, 1]',
    placeholder='Range',
    options=ranges,
    description='Normalization range',
    ensure_option=True,
    disabled=False,
    # layout={'width': 'max-content'},
    style={'description_width': 'initial'}
)

per_channel_mask = widgets.Checkbox(True,
                       description='Per channel normalization',
                       style={'description_width': 'initial'}
)


percentile_min_mask = widgets.FloatSlider(description='min percentile', value=2., min=0., max=30., step=0.1,style={'description_width': 'initial'})
percentile_max_mask = widgets.FloatSlider(description='max percentile', value=99.8, min=70., max=100., step=0.1,style={'description_width': 'initial'})
clip_percentile_mask_checkbox = widgets.Checkbox(
    value=False,
    description='Clip',
    disabled=False,
    indent=False,
    style={'description_width': 'initial'}
)

percentile_min_mask.layout.visibility = 'hidden'
percentile_max_mask.layout.visibility = 'hidden'
clip_percentile_mask_checkbox.layout.visibility = 'hidden'


def display_percentile_range_mask(x):
  if not 'ercent' in normalization_methods_mask.value:
    percentile_min_mask.layout.visibility = 'hidden'
    percentile_max_mask.layout.visibility = 'hidden'
    clip_percentile_mask_checkbox.layout.visibility = 'hidden'
    normalization_range_mask.layout.visibility = 'visible'
  else:
    percentile_min_mask.layout.visibility = 'visible'
    percentile_max_mask.layout.visibility = 'visible'
    clip_percentile_mask_checkbox.layout.visibility = 'visible'
    normalization_range_mask.layout.visibility = 'hidden'


interactive(display_percentile_range_mask, x=normalization_methods_mask);

remove_n_border_mask_pixels = widgets.IntSlider(description='Remove border pixels', value=0, min=0, max=10, step=1,style={'description_width': 'initial'})
mask_dilations = widgets.IntSlider(description='Dilation', value=0, min=0, max=10, step=1,style={'description_width': 'initial'})


# TODO add options to create and save temp files to gain time for training only if retraining a model
# or offer it all the time but tick it only if model need be retrained by default --> better option in case the user reloads things
# need make sure the model has seven outputs --> it is compatible with EPySeg

set_generate_epyseg_output = False
if model.value == 3:
  set_generate_epyseg_output = True


generate_default_epyseg_output_from_mask = widgets.Checkbox(set_generate_epyseg_output,
                       description='(EPySeg pre-trained model only!) Produce EPySeg-style output (from user input watershed mask)',
                       style={'description_width': 'initial'}
)
store_mask_on_drive_to_gain_speed = widgets.Checkbox(set_generate_epyseg_output,
                       description='Save EPySeg-style output on disk (if ticked, much faster, but disk space required)',
                       style={'description_width': 'initial'}
)

model_outputs = deepTA.get_outputs_shape()

# print(model_outputs[0][-1])
# print(model_outputs[0][-1]!= 7)
# model_outputs = None
# print(model_outputs)
# print(model.value == 3)

if model.value == 3 or (model_outputs is not None and model_outputs[0][-1] == 7):
  # ui = widgets.VBox([info_label_mask, channel_COI_mask, rule_reduc_mask, rule_aug_mask, normalization_methods_mask, normalization_range_mask, per_channel_mask, remove_n_border_mask_pixels,mask_dilations, generate_default_epyseg_output_from_mask, store_mask_on_drive_to_gain_speed])
  ui = widgets.VBox([info_label_mask, channel_COI_mask, rule_reduc_mask, rule_aug_mask,  widgets.HBox([normalization_methods_mask, normalization_range_mask, per_channel_mask]),widgets.HBox([percentile_min_mask, percentile_max_mask, clip_percentile_mask_checkbox]), remove_n_border_mask_pixels,mask_dilations, generate_default_epyseg_output_from_mask, store_mask_on_drive_to_gain_speed])    
else:
  # ui = widgets.VBox([info_label_mask, channel_COI_mask, rule_reduc_mask, rule_aug_mask, normalization_methods_mask, normalization_range_mask, per_channel_mask, remove_n_border_mask_pixels,mask_dilations])
  ui = widgets.VBox([info_label_mask, channel_COI_mask, rule_reduc_mask, rule_aug_mask,  widgets.HBox([normalization_methods_mask, normalization_range_mask, per_channel_mask]),widgets.HBox([percentile_min_mask, percentile_max_mask, clip_percentile_mask_checkbox]), remove_n_border_mask_pixels,mask_dilations])    

display(ui)



# Step 16
- Please run the cell below to set training parameters

In [16]:
#@title <-- Press Run


# input_normalization = {'method': 'Rescaling (min-max normalization)', 'range': [0, 1],
                          #  'individual_channels': True}

# now the batch size and alike parameters
batch_size_slider = widgets.IntSlider(description='Batch size', value=16, min=1, max=256, step=1,style={'description_width': 'initial'})
epochs_slider = widgets.IntSlider(description='Number of epochs', value=100, min=5, max=1000, step=1,style={'description_width': 'initial'})
keep_slider = widgets.IntSlider(description='Keep best models', value=5, min=-1, max=1000, step=1,style={'description_width': 'initial'})
# assume full set at every step

# add reduce lr on plateau
reduce_lr_on_plateau_checkbox = widgets.Checkbox(
    value=True,
    description='Reduce learning rate on plateau',
    disabled=False,
    indent=False,
    style={'description_width': 'initial'}
)
shuffle_checkbox = widgets.Checkbox(
    value=True,
    description='Shuffle training sets',
    disabled=False,
    indent=False,
    style={'description_width': 'initial'}
)

patience_slider = widgets.IntSlider(description='Patience', value=10, min=2, max=100, step=1,style={'description_width': 'initial'})
factor_slider = widgets.FloatSlider(description='Factor (0.5 means reduce lr by 2)', value=0.50, min=0.00, max=1.00, step=0.01,style={'description_width': 'initial'})
reduce_lr_on_plateau_box =  widgets.HBox([reduce_lr_on_plateau_checkbox, factor_slider, patience_slider,])

def show_hide_reduce_lr(x):
      # print(default_lr_checkbox.value)
      patience_slider.disabled = not reduce_lr_on_plateau_checkbox.value
      factor_slider.disabled = not reduce_lr_on_plateau_checkbox.value
      if not reduce_lr_on_plateau_checkbox.value:
        patience_slider.layout.visibility = 'hidden'
        factor_slider.layout.visibility = 'hidden'
      else:
        patience_slider.layout.visibility = 'visible'
        factor_slider.layout.visibility = 'visible'

interactive(show_hide_reduce_lr, x=reduce_lr_on_plateau_checkbox);

wdgts = [shuffle_checkbox, batch_size_slider, epochs_slider, keep_slider, reduce_lr_on_plateau_box]
ui = widgets.VBox(wdgts)
display(ui)

# Step 17
- Please run the cell to set the tiling parameters (to reduce memory usage)

In [17]:
#@title <-- Press Run

test_boolean = True # True # I can have a conditional GUI in fact
wdgts = []

input_shape = deepTA.get_inputs_shape()
output_shape = deepTA.get_outputs_shape()

# so far assume model has one entry and one output maybe allow several inputs and outputs later
# print('model input shape', input_shape[0])
# print('model output shape', output_shape[0])

input_val_width = 128
input_val_height = 128
if input_shape[0][-2] is not None:
  # print(input_shape[0][-2]) # default tile width
  input_val_width=input_shape[0][-2]
  test_boolean = False
if input_shape[0][-3] is not None:
  # print(input_shape[0][-3]) # default tile height
  input_val_height=input_shape[0][-3]
  test_boolean = False

tile_width_slider = widgets.IntSlider(description='tile width', value=input_val_width, min=16, max=1024, step=2,style={'description_width': 'initial'})
wdgts.append(tile_width_slider)
tile_height_slider = widgets.IntSlider(description='tile height', value=input_val_height, min=16, max=1024, step=2,style={'description_width': 'initial'})
wdgts.append(tile_height_slider)

# now the batch size and alike parameters
if not test_boolean:
  tile_width_slider.disabled = True
  tile_height_slider.disabled = True
  printmd('<font color="green">input width and height is already defined with the model, please move on to next step</font>')
else:
  ui = widgets.VBox(wdgts)
  display(ui)


# Step 18
- Please run the cell below to set the data augmentation parameters

In [18]:
#@title <-- Press Run

# need be defined here to avoid erase if people relaunch stuff and allow repopulate it maybe change this
augmentations = []

from epyseg.deeplearning.augmentation.generators.data import DataGenerator
import json

# maybe make this dynamic ??? because I need add data augs --> just give it a try but dynamic would be good ideally
types_of_augmentations = list(DataGenerator.augmentation_types_and_ranges.keys())
# print(types_of_augmentations)
# faire un add et ajouter les augs à une liste puis la passer au truc suivant (est ce que je permet de changer les parametres ou pas???? peut etre une autre fois, je pense qu'il faut que ce truc soit dynamique avec du interact...)
augmentation_drop = widgets.Dropdown(
    value='None',
    placeholder='Data augmentation',
    options=types_of_augmentations,
    description='Data augmentation',
    ensure_option=True,
    disabled=False,
    # layout={'width': 'max-content'},
    style={'description_width': 'initial'}
)

# TODO need a add button that adds to current augs and maybe a reset one too
# need a list to have all the augs at once



list1 = widgets.Select(
    options=augmentations,
    description='Selected augmentations:',
    disabled=False,
    style={'description_width': 'initial'}
)

button0 = widgets.Button(
    description='Add augmentation',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Add',
    icon='check',
    style={'description_width': 'initial'}
)

remove = widgets.Button(description='Remove')
def on_remove_clicked(_):
      # "linking function with output"
      # with out:
          # what happens when we press the button
          # clear_output()
          # print('Something happens!')
          try:
            augmentations.remove(list1.value)
            list1.options = augmentations
          except:
            pass

remove.on_click(on_remove_clicked)

data_aug_range_rate = widgets.FloatSlider(description='range/rate', value=0.05, min=0, max=0.25, step=0.01,style={'description_width': 'initial'}, disabled = True)

def on_combobox_changed(x):
    default_value = DataGenerator.augmentation_types_and_ranges[augmentation_drop.value]
    show = default_value is not None
    data_aug_range_rate.disabled = not show
    if show:
      data_aug_range_rate.max = default_value[1]
      data_aug_range_rate.min = default_value[0]
      data_aug_range_rate.value = default_value[2]

interact(on_combobox_changed, x=augmentation_drop);

add = widgets.Button(description='Add')
def on_button_clicked(_):
          aug_as_text = ''
          type = augmentation_drop.value
          if type == 'None':
              aug_as_text = json.dumps({'type': None})
          else:
              if data_aug_range_rate.disabled:
                  aug_as_text= json.dumps({'type': type})
              else:
                # no need to send value if 0
                if data_aug_range_rate.value == 0:
                    aug_as_text = json.dumps({'type': None})
                aug_as_text = json.dumps({'type':type, 'value':data_aug_range_rate.value})


          augmentations.append(aug_as_text)
          list1.options = augmentations

# linking button and function together using a button's method
add.on_click(on_button_clicked)

rotate_interpolation_free_data_augmenter_checkbox_checkbox = widgets.Checkbox(
    value=True,
    description='Rotate/flip augmented output (interpolation free)',
    disabled=False,
    indent=False,
    style={'description_width': 'initial'}
)


ui = widgets.VBox([ list1, data_aug_range_rate, widgets.HBox([add,remove]), rotate_interpolation_free_data_augmenter_checkbox_checkbox])#augmentation_drop,
display(ui)


interactive(children=(Dropdown(description='Data augmentation', options=('None', 'shear', 'zoom', 'rotate', 'r…

# Step 19
- Please run the cell below to train the model

In [20]:
#@title <-- Press Run

train_parameters = {}
dataset= {}

items = []
# reconvert json string to dict
for aug in list1.options:
    items.append(json.loads(aug))
train_parameters['augmentations'] = items
train_parameters['rotate_n_flip_independently_of_augmentation'] = rotate_interpolation_free_data_augmenter_checkbox_checkbox.value

# train_parameters['input_normalization'] = {'method': normalization_methods_orig.value,
#                                            'individual_channels': per_channel_orig.value,
#                                            'range': normalization_range_orig.value}
range_input = normalization_range_orig.value
if percentile_min_orig.layout.visibility == 'visible':
    # TODO need a check that values are ok --> but must be ok because I bounded them and bounds are non overlapping
    range_input=[percentile_min_orig.value, percentile_max_orig.value]
train_parameters['input_normalization'] = {'method': normalization_methods_orig.value,
                                          'individual_channels': per_channel_orig.value,
                                          'range': range_input,
                                          'clip': True if clip_percentile_orig_checkbox.value and clip_percentile_orig_checkbox.layout.visibility == 'visible' else False} # can have different values for the range
# train_parameters['output_normalization'] = {'method': normalization_methods_mask.value,
#                                            'individual_channels': per_channel_mask.value,
#                                            'range': normalization_range_mask.value}                                           
range_output = normalization_range_orig.value
if percentile_min_mask.layout.visibility == 'visible':
    # TODO need a check that values are ok --> but must be ok because I bounded them and bounds are non overlapping
    range_output=[percentile_min_mask.value, percentile_max_mask.value]
train_parameters['output_normalization'] = {'method': normalization_methods_mask.value,
                                          'individual_channels': per_channel_mask.value,
                                          'range': range_output,
                                          'clip': True if clip_percentile_mask_checkbox.value and clip_percentile_mask_checkbox.layout.visibility == 'visible' else False} # can have different values for the range

dataset['inputs'] = [folder_train_originals.path]
dataset['outputs'] = [folder_train_segmentation.path]

def get_clip_by_freq():
      if 'ignore' in clipping_methods_orig.value:
          return {'lower_cutoff': None, 'upper_cutoff': None,
                  'channel_mode': per_channel_orig.value}
      elif clipping_methods_orig.value == '+':
          return {'lower_cutoff': None, 'upper_cutoff': cliprange_orig.value,
                  'channel_mode': per_channel_orig.value}
      elif clipping_methods_orig.value == '-':
          return {'lower_cutoff': cliprange_orig.value, 'upper_cutoff': None,
                  'channel_mode': per_channel_orig.value}
      else:
        #  '/' in clipping_methods_orig.value:
          return {'lower_cutoff': cliprange_orig.value, 'upper_cutoff': cliprange_orig.value,
                  'channel_mode': per_channel_orig.value}

train_parameters['clip_by_frequency'] = get_clip_by_freq()

dataset['input_channel_reduction_rule'] =rule_reduc_orig.value
dataset['input_channel_augmentation_rule'] =rule_aug_orig.value
dataset['input_channel_of_interest'] =channel_COI_orig.value
dataset['output_channel_reduction_rule'] =rule_reduc_mask.value
dataset['output_channel_augmentation_rule'] =rule_aug_mask.value
dataset['output_channel_of_interest'] =channel_COI_mask.value
train_parameters['epochs'] = epochs_slider.value
train_parameters['keep_n_best'] = keep_slider.value
train_parameters['steps_per_epoch'] = -1 # always full set maybe change this some day though
train_parameters['shuffle'] = shuffle_checkbox.value
train_parameters['batch_size'] = batch_size_slider.value
train_parameters['batch_size_auto_adjust'] = True
train_parameters['validation_split'] = 0
dataset['crop_parameters'] = None #TODO maybe some day...
dataset['remove_n_border_mask_pixels'] = remove_n_border_mask_pixels.value
dataset['mask_dilations'] = mask_dilations.value
dataset['invert'] = invert_orig.value
if generate_default_epyseg_output_from_mask.value:
  if store_mask_on_drive_to_gain_speed.value:
      dataset['create_epyseg_style_output'] = 'sevenmaskssave'
  else:
      dataset['create_epyseg_style_output'] = 'sevenmasks'

train_parameters['datasets'] = [dataset]

train_parameters['default_input_tile_width'] = tile_width_slider.value
train_parameters['default_input_tile_height'] = tile_height_slider.value
train_parameters['default_output_tile_width'] = tile_width_slider.value
train_parameters['default_output_tile_height'] = tile_height_slider.value

train_parameters['lr'] = None if default_lr_checkbox.value else learning_rate_spinner.value
train_parameters['reduce_lr_on_plateau'] = None if not reduce_lr_on_plateau_checkbox.value else factor_slider.value
train_parameters['patience'] = patience_slider.value

if False:
  print(train_parameters)
else:
# 'datasets': --> add inputs and outputs there

# if True:

  batch_size = batch_size_slider.value #16 # TODO specify that here too
  NB_EPOCHS = epochs_slider.value #180  # 80 # 100 # 10 # 150
  deepTA.get_loaded_model_params()
  metaAugmenter = MetaAugmenter(input_shape=input_shape, output_shape=output_shape, **train_parameters)

  metaAugmenter.appendDatasets(**train_parameters)
  deepTA.train(metaAugmenter, progress_callback=None, **train_parameters)

  print('Saving model')
  # TODO save model here as pb and as JSON, pb is better as it has the compiler, etc...
  deepTA.saveModel()
  deepTA.saveAsJsonWithWeights()
  # deepTA.plot_graph(deepTA.model._name + '_graph.png')
  print('Done')


INFO - 2023-05-04 09:56:49,028 - deepl.py - train - line 1165 - train dataset batches: 6
validation dataset batches: 0

INFO:master:train dataset batches: 6
validation dataset batches: 0
INFO - 2023-05-04 09:56:49,031 - deepl.py - train - line 1191 - Reduce learning rate on plateau is enabled.

INFO:master:Reduce learning rate on plateau is enabled.
INFO - 2023-05-04 09:56:49,035 - deepl.py - train - line 1195 - Reduce learning rate is monitoring "loss"

INFO:master:Reduce learning rate is monitoring "loss"


3.90625e-06


INFO - 2023-05-04 09:56:49,130 - saver.py - on_epoch_begin - line 109 - 0.0%

INFO:master:0.0%


Epoch 1/100
6/6 [==============================] - ETA: 0s - loss: 0.9440 - iou_score: 0.0560

INFO - 2023-05-04 09:56:51,008 - saver.py - on_epoch_end - line 128 - Estimated remaining run time: 3.1298068316666177 minute(s)

INFO:master:Estimated remaining run time: 3.1298068316666177 minute(s)
INFO - 2023-05-04 09:56:51,011 - saver.py - on_epoch_end - line 368 - saving file: Linknet-vgg16-sigmoid-0.h5

INFO:master:saving file: Linknet-vgg16-sigmoid-0.h5


...

INFO - 2023-05-04 09:58:52,197 - saver.py - on_epoch_begin - line 109 - 99.0%

INFO:master:99.0%


Epoch 100/100
6/6 [==============================] - ETA: 0s - loss: 0.9423 - iou_score: 0.0577

INFO - 2023-05-04 09:58:53,277 - saver.py - on_epoch_end - line 131 - Estimated remaining run time: 1.0808194349999667 second(s)

INFO:master:Estimated remaining run time: 1.0808194349999667 second(s)



Epoch 100: ReduceLROnPlateau reducing learning rate to 7.629394893626795e-09.
6/6 [==============================] - 1s 201ms/step - loss: 0.9423 - iou_score: 0.0577 - lr: 1.5259e-08
Saving model


Done
